In [2]:
#QUESTION1
import requests
import zipfile
import io
import os

# URL for the Dakshina dataset
url = "https://github.com/google-research-datasets/dakshina/archive/refs/heads/master.zip"

# Directory to save the dataset
dataset_dir = "dakshina_dataset"

# Create the directory if it doesn't exist
os.makedirs(dataset_dir, exist_ok=True)

# Download the dataset
response = requests.get(url)

# Check if the download was successful
if response.status_code == 200:
    # Extract the dataset
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(dataset_dir)
    print(f"Dakshina dataset downloaded and extracted to '{dataset_dir}'")
else:
    print(f"Failed to download the dataset: {response.status_code} {response.reason}")


Dakshina dataset downloaded and extracted to 'dakshina_dataset'


In [3]:

from google.colab import files

# Upload your dataset
uploaded = files.upload()


Saving hi.translit.sampled.train.tsv to hi.translit.sampled.train.tsv


In [4]:
import os

# List all files in the current directory (content)
print(os.listdir("/content"))

['.config', 'hi.translit.sampled.train.tsv', 'dakshina_dataset', 'sample_data']


In [5]:
import pandas as pd

# Define the path to the uploaded file
lexicon_path = "/content/hi.translit.sampled.train.tsv"

# Load the file into a DataFrame
df = pd.read_csv(lexicon_path, sep='\t', header=None, names=['native', 'latin', 'attested'])

# Show the first 5 rows
print(df.head())


    native     latin  attested
0       अं        an         3
1  अंकगणित  ankganit         3
2     अंकल     uncle         4
3    अंकुर     ankur         4
4   अंकुरण   ankuran         3


In [6]:
import numpy as np
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Hyperparameters
batch_size = 64
epochs = 150
latent_dim = 256
max_samples = 50000

# Path to Dakshina dataset
file_path = "/content/hi.translit.sampled.train.tsv"

# Load and preprocess data
input_texts, target_texts = [], []
input_chars, target_chars = set(), set()

with open(file_path, encoding="utf-8") as f:
    lines = f.read().splitlines()

for line in lines[:min(max_samples, len(lines) - 1)]:
    native, latin, _ = line.split("\t")
    input_seq = latin.strip()
    target_seq = "\t" + native.strip() + "\n"

    input_texts.append(input_seq)
    target_texts.append(target_seq)

    input_chars.update(set(input_seq))
    target_chars.update(set(target_seq))

input_chars.add(" ")
target_chars.add(" ")

input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))

input_token_index = {ch: i for i, ch in enumerate(input_chars)}
target_token_index = {ch: i for i, ch in enumerate(target_chars)}

num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)

max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

# Prepare one-hot encoded data
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

for i, (inp_txt, tgt_txt) in enumerate(zip(input_texts, target_texts)):
    for t, ch in enumerate(inp_txt):
        encoder_input_data[i, t, input_token_index[ch]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[" "]] = 1.

    for t, ch in enumerate(tgt_txt):
        decoder_input_data[i, t, target_token_index[ch]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[ch]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[" "]] = 1.
    decoder_target_data[i, t:, target_token_index[" "]] = 1.

# Encoder setup
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder setup
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Full training model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

# Training
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)

# Save model
model.save("s2s_model.keras")


Epoch 1/150
553/553 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.6884 - loss: 1.3217 - val_accuracy: 0.7318 - val_loss: 1.1070
Epoch 2/150
553/553 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.7682 - loss: 0.8638 - val_accuracy: 0.7445 - val_loss: 1.0638
Epoch 3/150
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7975 - loss: 0.7312 - val_accuracy: 0.7479 - val_loss: 1.0854
Epoch 4/150
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8148 - loss: 0.6557 - val_accuracy: 0.7530 - val_loss: 1.0897
Epoch 5/150
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8310 - loss: 0.5893 - val_accuracy: 0.7555 - val_loss: 1.0799
Epoch 6/150
553/553 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8437 - loss: 0.5352 - val_accuracy: 0.7575 - val_loss: 1.0768
Epoch 7/150
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8592 - loss: 0.4728 - val_accuracy: 0.7682 - val_loss: 0.9972
Epoch 8/150
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8762 - loss: 0.4056 - val_acc

In [9]:
import numpy as np
from keras.models import load_model, Model
from keras.layers import Input

# Load the trained seq2seq model
model = load_model("s2s_model.keras")

# Define encoder model
encoder_input = model.input[0]  # First input is encoder input
encoder_outputs, state_h, state_c = model.layers[2].output  # LSTM layer
encoder_states = [state_h, state_c]
encoder_model = Model(encoder_input, encoder_states)

# Define decoder model
decoder_input = model.input[1]  # Second input is decoder input
latent_dim = state_h.shape[-1]  # Extract latent dimension

# Define new input layers for the decoder states
decoder_input_h = Input(shape=(latent_dim,), name="decoder_input_h")
decoder_input_c = Input(shape=(latent_dim,), name="decoder_input_c")
decoder_states_inputs = [decoder_input_h, decoder_input_c]

# Reuse LSTM and Dense layers from original model
decoder_lstm = model.layers[3]
decoder_dense = model.layers[4]

# Connect decoder layers
lstm_outputs, new_h, new_c = decoder_lstm(decoder_input, initial_state=decoder_states_inputs)
decoder_states = [new_h, new_c]
decoder_outputs = decoder_dense(lstm_outputs)

# Build final decoder model
decoder_model = Model(
    [decoder_input] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Reverse dictionaries for decoding
reverse_input_char_index = {idx: char for char, idx in input_token_index.items()}
reverse_target_char_index = {idx: char for char, idx in target_token_index.items()}

# Decoder function
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq, verbose=0)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    decoded_text = ""
    for _ in range(max_decoder_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_text += sampled_char

        if sampled_char == "\n":
            break

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]

    return decoded_text.strip()

# Example usage:
# transliteration = decode_sequence(input_seq)
# print("Devanagari transliteration:", transliteration)


In [10]:
import numpy as np

# Define constants based on your training configuration
max_encoder_seq_length = 15
max_decoder_seq_length = 20

# Ensure these are already loaded before this script is run:
# input_token_index, target_token_index, reverse_target_char_index, encoder_model, decoder_model

num_encoder_tokens = len(input_token_index)
num_decoder_tokens = len(target_token_index)

# Function to encode input text into one-hot encoded format for the encoder
def encode_input_text(input_text):
    encoder_input = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype="float32")
    for t, char in enumerate(input_text):
        if char in input_token_index:
            encoder_input[0, t, input_token_index[char]] = 1.0
        else:
            print(f"Warning: character '{char}' not in training vocabulary.")
    for t in range(len(input_text), max_encoder_seq_length):
        encoder_input[0, t, input_token_index[' ']] = 1.0
    return encoder_input

# Function to decode the predicted sequence from the decoder
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq, verbose=0)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    decoded_sentence = ""
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            break

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]

    return decoded_sentence.strip()

# Command-line interface to test transliteration interactively
def translate_latin_to_devanagari():
    print("Enter a Latin word to transliterate (type 'exit' to quit):")
    while True:
        user_input = input("Latin word: ").strip()
        if user_input.lower() == "exit":
            break

        input_seq = encode_input_text(user_input)
        output = decode_sequence(input_seq)
        print("Devanagari transliteration:", output)

# Start interactive mode
translate_latin_to_devanagari()


Enter a Latin word to transliterate (type 'exit' to quit):
Latin word: yuktha
Devanagari transliteration: युक्ता
Latin word: priya
Devanagari transliteration: प्रिया
Latin word: chinmayee
Devanagari transliteration: चिनामेयर
Latin word: abhi
Devanagari transliteration: अभी
Latin word: siddu
Devanagari transliteration: एसडीड्स
Latin word: sushrut
Devanagari transliteration: स्षूष्ट्र
Latin word: vardhan
Devanagari transliteration: वर्धन
Latin word: harini
Devanagari transliteration: अरिनी
Latin word: bhavya
Devanagari transliteration: भाव्य
Latin word: divya
Devanagari transliteration: दिव्या
Latin word: teju
Devanagari transliteration: तेजू
Latin word: vaagdevi
Devanagari transliteration: वादविद्व
Latin word: rishitha
Devanagari transliteration: ऋषिवती
Latin word: vilekha
Devanagari transliteration: निवेलका
Latin word: taranya
Devanagari transliteration: तरणय
Latin word: sudheer
Devanagari transliteration: क्षूधरी
Latin word: kumudhwathi
Devanagari transliteration: कुमुन्धवेश
Latin wor